# Modelo e Submissão para Kaggle
## Jigsaw Toxic Comment Classification Challenge

Este notebook treina um modelo e gera o arquivo de submissão para o Kaggle.

## 1. Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier

# Processamento de texto
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

print("Bibliotecas importadas com sucesso!")

# Baixar recursos do NLTK (se necessário)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)

Bibliotecas importadas com sucesso!


## 2. Carregamento dos Dados

In [2]:
# Definir caminhos
data_dir = Path('../data/raw')

# Carregar dados
print("Carregando dados de treino...")
train_df = pd.read_csv(data_dir / 'train.csv')
print(f"Shape: {train_df.shape}")

print("\nCarregando dados de teste...")
test_df = pd.read_csv(data_dir / 'test.csv')
print(f"Shape: {test_df.shape}")

print("\nCarregando sample submission...")
sample_submission = pd.read_csv(data_dir / 'sample_submission.csv')
print(f"Shape: {sample_submission.shape}")

# Colunas de toxicidade
toxic_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print(f"\nClasses de toxicidade: {toxic_cols}")

Carregando dados de treino...
Shape: (159571, 8)

Carregando dados de teste...
Shape: (153164, 2)

Carregando sample submission...
Shape: (153164, 7)

Classes de toxicidade: ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


## 3. Pré-processamento de Texto

In [3]:
def preprocess_text(text):
    """
    Pré-processa o texto:
    - Remove URLs
    - Remove caracteres especiais
    - Converte para minúsculas
    - Remove espaços extras
    """
    if pd.isna(text):
        return ""
    
    # Converter para string
    text = str(text)
    
    # Remover URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remover caracteres especiais, manter apenas letras, números e espaços
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Converter para minúsculas
    text = text.lower()
    
    # Remover espaços extras
    text = ' '.join(text.split())
    
    return text

# Aplicar pré-processamento
print("Pré-processando textos de treino...")
train_df['comment_text_processed'] = train_df['comment_text'].apply(preprocess_text)

print("Pré-processando textos de teste...")
test_df['comment_text_processed'] = test_df['comment_text'].apply(preprocess_text)

print("\nExemplo de texto original:")
print(train_df['comment_text'].iloc[0][:200])
print("\nExemplo de texto processado:")
print(train_df['comment_text_processed'].iloc[0][:200])

Pré-processando textos de treino...
Pré-processando textos de teste...

Exemplo de texto original:
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove th

Exemplo de texto processado:
explanation why the edits made under my username hardcore metallica fan were reverted they werent vandalisms just closure on some gas after i voted at new york dolls fac and please dont remove the tem


## 4. Feature Engineering - TF-IDF

In [4]:
# Criar vetorizador TF-IDF
# Limitar o número de features para não consumir muita memória
max_features = 5000

print(f"Criando vetorizador TF-IDF com max_features={max_features}...")
vectorizer = TfidfVectorizer(
    max_features=max_features,
    ngram_range=(1, 2),  # Unigramas e bigramas
    min_df=2,  # Palavra deve aparecer em pelo menos 2 documentos
    max_df=0.95,  # Palavra não deve aparecer em mais de 95% dos documentos
    stop_words='english'
)

# Treinar o vetorizador e transformar dados de treino
print("\nTreinando vetorizador e transformando dados de treino...")
X_train = vectorizer.fit_transform(train_df['comment_text_processed'])
print(f"Shape dos dados de treino: {X_train.shape}")

# Transformar dados de teste
print("\nTransformando dados de teste...")
X_test = vectorizer.transform(test_df['comment_text_processed'])
print(f"Shape dos dados de teste: {X_test.shape}")

# Labels
y_train = train_df[toxic_cols].values
print(f"\nShape dos labels: {y_train.shape}")

Criando vetorizador TF-IDF com max_features=5000...

Treinando vetorizador e transformando dados de treino...
Shape dos dados de treino: (159571, 5000)

Transformando dados de teste...
Shape dos dados de teste: (153164, 5000)

Shape dos labels: (159571, 6)


## 5. Divisão dos Dados (Treino/Validação)

In [5]:
# Dividir dados de treino em treino e validação
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, 
    test_size=0.2, 
    random_state=42,
    stratify=None  # Não estratificar porque temos múltiplas classes
)

print(f"Dados de treino: {X_train_split.shape}")
print(f"Dados de validação: {X_val_split.shape}")

Dados de treino: (127656, 5000)
Dados de validação: (31915, 5000)


## 6. Treinamento do Modelo

In [6]:
# Usar OneVsRestClassifier com LogisticRegression
# Isso treina um classificador binário para cada classe
print("Treinando modelo (OneVsRest + LogisticRegression)...")
print("Isso pode levar alguns minutos...")

model = OneVsRestClassifier(
    LogisticRegression(
        max_iter=1000,
        random_state=42,
        solver='lbfgs',
        C=1.0
    ),
    n_jobs=-1  # Usar todos os cores disponíveis
)

# Treinar o modelo
model.fit(X_train_split, y_train_split)

print("\nModelo treinado com sucesso!")

Treinando modelo (OneVsRest + LogisticRegression)...
Isso pode levar alguns minutos...

Modelo treinado com sucesso!


## 7. Avaliação do Modelo

In [7]:
# Fazer previsões no conjunto de validação
print("Fazendo previsões no conjunto de validação...")
y_val_pred = model.predict_proba(X_val_split)

# Calcular ROC-AUC para cada classe
print("\n=== Métricas de Validação ===")
scores = {}
for i, col in enumerate(toxic_cols):
    score = roc_auc_score(y_val_split[:, i], y_val_pred[:, i])
    scores[col] = score
    print(f"{col:20s}: {score:.4f}")

# Média dos scores
mean_score = np.mean(list(scores.values()))
print(f"\n{'Média':20s}: {mean_score:.4f}")
print("\n(Quanto maior, melhor. Score máximo é 1.0)")

Fazendo previsões no conjunto de validação...

=== Métricas de Validação ===
toxic               : 0.9586
severe_toxic        : 0.9758
obscene             : 0.9772
threat              : 0.9812
insult              : 0.9688
identity_hate       : 0.9561

Média               : 0.9696

(Quanto maior, melhor. Score máximo é 1.0)


## 8. Previsões no Conjunto de Teste

In [8]:
# Treinar modelo final com todos os dados de treino
print("Treinando modelo final com todos os dados de treino...")
model_final = OneVsRestClassifier(
    LogisticRegression(
        max_iter=1000,
        random_state=42,
        solver='lbfgs',
        C=1.0
    ),
    n_jobs=-1
)

model_final.fit(X_train, y_train)
print("Modelo final treinado!")

# Fazer previsões no conjunto de teste
print("\nFazendo previsões no conjunto de teste...")
test_predictions = model_final.predict_proba(X_test)
print(f"Shape das previsões: {test_predictions.shape}")

Treinando modelo final com todos os dados de treino...
Modelo final treinado!

Fazendo previsões no conjunto de teste...
Shape das previsões: (153164, 6)


## 9. Criar Arquivo de Submissão

In [9]:
# Criar DataFrame de submissão
submission = pd.DataFrame({
    'id': test_df['id']
})

# Adicionar previsões para cada classe
for i, col in enumerate(toxic_cols):
    submission[col] = test_predictions[:, i]

# Verificar formato
print("=== Formato da Submissão ===")
print(f"Shape: {submission.shape}")
print(f"\nPrimeiras linhas:")
display(submission.head())

print(f"\nEstatísticas das previsões:")
display(submission[toxic_cols].describe())

# Verificar se está no formato correto
print(f"\nColunas esperadas: {list(sample_submission.columns)}")
print(f"Colunas criadas: {list(submission.columns)}")
assert list(submission.columns) == list(sample_submission.columns), "Colunas não correspondem!"
print("\n✓ Formato correto!")

=== Formato da Submissão ===
Shape: (153164, 7)

Primeiras linhas:


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.995305,0.147074,0.994955,0.026196,0.923182,0.271023
1,0000247867823ef7,0.008492,0.003082,0.006626,0.001479,0.007976,0.003164
2,00013b17ad220c46,0.017330,0.001493,0.005017,0.000777,0.006776,0.001579
3,00017563c3f7919a,0.005377,0.002018,0.003994,0.000880,0.003533,0.001033
4,00017695ad8997eb,0.061986,0.002646,0.014106,0.001266,0.021166,0.002553



Estatísticas das previsões:


,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,153164.000000,153164.000000,153164.000000,153164.000000,153164.000000,153164.000000
mean,0.192169,0.017377,0.113866,0.004982,0.096087,0.016878
std,0.317059,0.067973,0.256959,0.033443,0.214843,0.074337
min,0.000036,0.000045,0.000187,0.000093,0.000069,0.000037
25%,0.009991,0.001710,0.006272,0.000906,0.005252,0.001641
50%,0.029951,0.003024,0.012770,0.001342,0.012456,0.003225
75%,0.171244,0.005669,0.036087,0.001970,0.042947,0.006409
max,1.000000,0.993290,1.000000,0.984673,0.999997,0.998147



Colunas esperadas: ['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
Colunas criadas: ['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

✓ Formato correto!


## 10. Salvar Arquivo de Submissão

In [10]:
# Criar diretório para submissões
submission_dir = Path('../submissions')
submission_dir.mkdir(exist_ok=True)

# Salvar arquivo de submissão
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'submission_{timestamp}.csv'
filepath = submission_dir / filename

submission.to_csv(filepath, index=False)
print(f"Arquivo de submissão salvo em: {filepath}")
print(f"Tamanho do arquivo: {filepath.stat().st_size / 1024:.2f} KB")

# Também salvar como submission.csv (mais fácil de encontrar)
submission.to_csv(submission_dir / 'submission.csv', index=False)
print(f"\nTambém salvo como: {submission_dir / 'submission.csv'}")

Arquivo de submissão salvo em: ..\submissions\submission_20260121_192044.csv
Tamanho do arquivo: 21492.82 KB

Também salvo como: ..\submissions\submission.csv


## 11. Instruções para Submeter no Kaggle

### Passo a Passo para Submeter no Kaggle:

1. **Acesse a página da competição:**
   - https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

2. **Vá para a aba "Submit Predictions"** (ou "Submeter Previsões")

3. **Clique em "Upload Submission File"**

4. **Selecione o arquivo:**
   - `submissions/submission.csv` ou o arquivo com timestamp

5. **Clique em "Make Submission"**

6. **Aguarde o processamento** (pode levar alguns minutos)

7. **Veja seu score!** O Kaggle mostrará sua pontuação (ROC-AUC médio)

### Dicas:
- Você pode submeter até 5 vezes por dia
- O score público é calculado em uma parte do conjunto de teste
- O score privado (final) é calculado após o encerramento da competição
- Tente melhorar o modelo e submeter novamente!

## 13. Conclusão e Trabalhos Futuros

### Resultados Obtidos

Este projeto desenvolveu com sucesso um modelo de classificação multirrótulo capaz de identificar comentários tóxicos com um score ROC-AUC médio de **0.9696 (96.96%)** no conjunto de validação. A solução utiliza técnicas consolidadas de NLP (TF-IDF) combinadas com algoritmos de Machine Learning (Regressão Logística), demonstrando eficácia na tarefa proposta.

### Trabalhos Futuros

Para melhorar ainda mais o desempenho do modelo, as seguintes abordagens podem ser exploradas:

1. **Modelos de Deep Learning**:
   - Implementação de redes neurais recorrentes (LSTM, GRU) para capturar dependências temporais
   - Fine-tuning de modelos pré-treinados como **BERT**, **DistilBERT** ou **RoBERTa** para aproveitar conhecimento semântico avançado
   - Arquiteturas Transformer para processamento de sequências longas

2. **Feature Engineering Avançado**:
   - Incorporação de word embeddings pré-treinados (Word2Vec, GloVe, FastText)
   - Aumento do número de features TF-IDF e exploração de n-gramas maiores (trigramas)
   - Extração de features linguísticas adicionais (comprimento do texto, densidade de palavras-chave, etc.)

3. **Otimização de Hiperparâmetros**:
   - Utilização de técnicas como GridSearch ou RandomizedSearch para encontrar os melhores parâmetros
   - Implementação de validação cruzada para avaliação mais robusta
   - Ajuste fino do parâmetro de regularização C na Regressão Logística

4. **Tratamento de Classes Desbalanceadas**:
   - Aplicação de técnicas como SMOTE para balanceamento sintético
   - Uso de class weights para dar mais importância a classes minoritárias
   - Implementação de Focal Loss para focar em exemplos difíceis de classificar

5. **Ensemble de Modelos**:
   - Combinação de múltiplos modelos (XGBoost, LightGBM, CatBoost) através de voting ou stacking
   - Integração de diferentes representações de texto (TF-IDF + Word Embeddings)

### Considerações Finais

A solução apresentada demonstra uma abordagem sistemática e bem fundamentada para o problema de classificação de comentários tóxicos, utilizando técnicas estabelecidas de Machine Learning. O modelo desenvolvido é eficiente, interpretável e alcança resultados competitivos, servindo como uma base sólida para futuras melhorias e explorações de técnicas mais avançadas.

---

**Bibliotecas Utilizadas:**
- pandas, numpy: Manipulação de dados
- scikit-learn: Machine Learning (TfidfVectorizer, LogisticRegression, OneVsRestClassifier)
- nltk: Processamento de linguagem natural
- re: Expressões regulares

### Ideias para melhorar a pontuação:

1. **Pré-processamento mais avançado:**
   - Lemmatização/Stemming
   - Remoção de stopwords mais inteligente
   - Tratamento de emojis e caracteres especiais

2. **Feature Engineering:**
   - Aumentar max_features do TF-IDF
   - Usar n-gramas maiores (trigramas)
   - Adicionar features de comprimento do texto
   - Word embeddings (Word2Vec, GloVe, FastText)

3. **Modelos mais avançados:**
   - XGBoost, LightGBM, CatBoost
   - Redes Neurais (LSTM, BERT, DistilBERT)
   - Ensemble de modelos

4. **Otimização de hiperparâmetros:**
   - GridSearch ou RandomizedSearch
   - Validação cruzada

5. **Tratamento de classes desbalanceadas:**
   - SMOTE
   - Class weights
   - Focal Loss